In [1]:
options(warn = -1)
source("https://raw.githubusercontent.com/balthapaixao/Covid19_BR_underreport/master/Func_/new_preproc.R",)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘reshape’


The following object is masked from ‘package:dplyr’:

    rename


Loading required package: greybox

Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 

Package "greybox", v0.6.0 loaded.

This is package "smooth", v2.6.0

Loading required package: forecast


Attaching package: ‘ModelMetrics’


The following object is masked from ‘package:base’:

    kappa


randomForest 4.6-14

Type rfNews() to see new features/changes/bug fixes.


Attaching package: ‘randomForest’


The following object is masked from ‘package:dplyr’:

    combine


Loading required package: Rcpp


Attaching package: ‘RSNNS’


The following object is masked from ‘package:ModelMetrics’:

    confusionMatrix


Loading required package: KernelKn

# Preparing the data

In [2]:
#X <- srag #recent data acquired from Infogripe Gitlab
X <- read_csv('https://raw.githubusercontent.com/balthapaixao/Covid19_BR_underreport/master/Aux_arqs/srag_data_28_05.csv', col_types = cols()) #data used for the article

In [3]:
pre_proc_ms

function (datelim = "2020-05-02") 
{
    infogripe_data = datelim
    serie_ms <- read_delim("https://raw.githubusercontent.com/balthapaixao/Covid19_BR_underreport/master/Aux_arqs/HIST_PAINEL_COVIDBR_31mai2020.csv", 
        delim = ";", col_types = cols())
    serie_ms["X1"] <- NULL
    serie_ms$casosAcumulado <- as.numeric(serie_ms$casosAcumulado)
    serie_ms$obitosAcumulado <- as.numeric(serie_ms$obitosAcumulado)
    serie <- serie_ms[which(serie_ms$regiao != "Brasil" & serie_ms$data == 
        infogripe_data), ]
    serie_total_casos <- serie[c("estado", "casosAcumulado")]
    serie_total_obitos <- serie[c("estado", "obitosAcumulado")]
    serie_total_casos <- aggregate(serie_total_casos$casosAcumulado, 
        by = list(Category = serie_total_casos$estado), FUN = max)
    serie_total_obitos <- aggregate(serie_total_obitos$obitosAcumulado, 
        by = list(Category = serie_total_obitos$estado), FUN = max)
    serie_ms_total <- list(hm_acc_cases = serie_total_casos, 
        hm_acc_deaths = serie_total_obitos)
    return(serie_ms_total)
}

In [4]:
date = '2020-05-02'
#For choosing the date the format is 'YYYY-mm-dd',
#default date is 2020-05-02 - 18th Epidemiological Week
# the date must be the end of an EW 
#https://portalsinan.saude.gov.br/calendario-epidemiologico-2020

data <- pre_proc_data(X, "underreport", date) #Preparing the data for underreport analysis
hm_data <- pre_proc_ms(date)

## After pre_proc_data("underreport") the data can be acessed by

<a href="https://github.com/balthapaixao/Covid19_BR_underreport/blob/master/Aux_arqs/dataflow_covid.png"><img src="https://i.ibb.co/hLwQJkp/dataflow-covid.png" alt="dataflow-covid" border="0"></a><br /><a target='_blank' href='https://github.com/balthapaixao/Covid19_BR_underreport/blob/master/Aux_arqs/dataflow_covid.png'>Dataflow Image </a><br />

In [5]:
#selecting cases series
serie <- data$deaths$no_covid
serie_covid <- data$deaths$covid

hm <- hm_data$hm_acc_deaths #Health Ministry data for rate_table

In [6]:
error_df <- calc_error(serie)
error_df

[1] "------- GENERATING THE ERRORS OF THE MODEL TABLE -------"


,epsilon_baseline,interval
,<dbl>,<fct>
AC,0.480,"[0.284, 0.683]"
AL,0.293,"[0.147, 0.452]"
AM,0.670,"[0.391, 1.075]"
AP,0.047,"[0.007, 0.102]"
BA,0.847,"[0.571, 1.142]"
CE,0.670,"[0.381, 1.107]"
DF,0.423,"[0.271, 0.618]"
ES,0.381,"[0.151, 0.661]"
GO,0.940,"[0.496, 1.454]"


In [7]:
rate_table <- calc_underreport(serie, serie_covid, hm)

[1] "------- GENERATING THE CUMULATIVE UNDERREPORTED TABLE -------"
[1] "------- P-VALUES TEST TABLE -------"
   random_noise_test reported_values_test
AC             FALSE                FALSE
AL              TRUE                 TRUE
AM              TRUE                 TRUE
AP              TRUE                FALSE
BA              TRUE                 TRUE
CE              TRUE                 TRUE
DF              TRUE                 TRUE
ES              TRUE                FALSE
GO              TRUE                FALSE
MA              TRUE                 TRUE
MG              TRUE                 TRUE
MS             FALSE                FALSE
MT              TRUE                FALSE
PA              TRUE                 TRUE
PB              TRUE                 TRUE
PE              TRUE                 TRUE
PI              TRUE                 TRUE
PR              TRUE                 TRUE
RJ              TRUE                 TRUE
RN              TRUE                 TRUE
RO      

In [8]:
print("------- GENERATING THE UNDER-REPORTING RATES TABLE -------")
rate_table

[1] "------- GENERATING THE UNDER-REPORTING RATES TABLE -------"


,Cum._Novelty_SARI,Cum._Covid_SARI,Rate,Cum._Covid_HM
,<dbl>,<dbl>,<chr>,<dbl>
AC,0,13,*,21
AL,49,34,0.441 +/- 0.029,58
AM,2023,1147,0.764 +/- 0.003,501
AP,26,21,**,40
BA,200,110,0.818 +/- 0.027,123
CE,1429,983,0.454 +/- 0.004,614
DF,90,33,1.727 +/- 0.061,31
ES,91,84,**,102
GO,61,36,**,30
